In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
from bson.json_util import dumps
import re

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "Password1"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#add the customers branding
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #create an anchor for the image/logo
    #make the image an href to the website, www.snhu.edu
    #open the link in a new tab by setting a blank target
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               height = 250, width = 251))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('Merrik Wright CS340 Project2))),
    html.Hr(),
    #create the radio buttons to act as a filter
    #set the default on initial load to to 'All'
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
        ],
        value='All'
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #allow a row to be selected
        selected_rows=[],
        filter_action="native", #allow a filter
        sort_action="native", #allow sorting
        page_action="native", #enable pagination
        page_current=0, #set start page
        page_size=10, #set rows per page

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content':'center'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    #set up an if/else if/else block to respond to the radio buttons
    if filter_type == 'All':
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({})) 
    elif filter_type == 'Water':

        #the regex patterns for the different filters
        labRegex = re.compile(".*lab.*", re.IGNORECASE)
        chesaRegex = re.compile(".*chesa.*", re.IGNORECASE)
        newRegex = re.compile(".*newf.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({
            '$or':[ #Regex isn't allowed in an $in helper so use $or
                {"breed": {'$regex': newRegex}}, #pass the regex to the filter
                {"breed": {'$regex': chesaRegex}},
                {"breed": {'$regex': labRegex}},
            ],
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    elif filter_type == 'Mountain':
        
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        alaskanRegex = re.compile(".*mala.*", re.IGNORECASE)
        oldRegex = re.compile(".*old engilish.*", re.IGNORECASE)
        huskyRegex = re.compile(".*husk.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({
            '$or':[
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': alaskanRegex}},
                {"breed": {'$regex': oldRegex}},
                {"breed": {'$regex': huskyRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    elif filter_type == 'Disaster':
        
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
        bloodRegex = re.compile(".*blood.*", re.IGNORECASE)
        doberRegex = re.compile(".*dober.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.getRecordCriteria({
            '$or':[
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': goldenRegex}},
                {"breed": {'$regex': bloodRegex}},
                {"breed": {'$regex': doberRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
    else:
        raise Exception("Unknown filter")
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')        
        
    return (data,columns)

    

    #All of the data for the color of a selected cell.
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]




# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
                       

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    #Code for chart. Started as pie chatt, but it was too much data. The histogram was much cleaner.
    dffP = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed',)
        )    
    ]

                       
                       
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    
    print(virtualRows)
    
    #Beaver Utah is located at 382.78181, -112.641131
    
    #The Views
    if not virtualRows: #A Default if no other line selected.
        markerArray = (38.278181, -112.641131) #Beaver Utah Animal Shelter
        toolTip = "Beaver Animal Shelter"
        popUpHeading = "Beaver Animal Shelter"
        popUpParagraph = "Shelter Location"
        
    else: #views based on the selection
        dff = pd.DataFrame(df.iloc[virtualRows]) #Conversion of the datatable
        coordLat = float(dff['location_lat'].to_string().split()[1]) #The lattitude
        coordLong = float(dff['location_long'].to_string().split()[1]) #The longitude
        markerArray = (coordLat, coordLong) # the array that is based on the selection above. 
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']
        
    return [d1.Map(style={'width': '600px', 'height': '300px'}, center=[38.278181, -112.641131],
                  zoom=10, children[d1.TileLayer(id="base-layer-id"),
                                                d1.Marker(position=markerArray, children=[
                                                    d1.Tooltip(toolTip),
                                                    d1.Popup([
                                                        html.H1(popUpHeading),
                                                        html.P(popUpParagraph)
                                                    ])
                                                ])
                                            ])
           ]
    
#FIXME Add in the code for your geolocation chart
    

app.run_server(debug=True)

SyntaxError: positional argument follows keyword argument (3711699291.py, line 113)